<h1>Person Re-Identification of favtGAN Generated Thermal Images using the `torchreid` Library</h1>
    
- Github Repo: https://github.com/KaiyangZhou/deep-person-reid

- Tech report: https://arxiv.org/abs/1910.10093

We used the `torchreid` library developed by Zhou Kaiyang which includes a variety of algorithms for person re-identification. We selected the MuDeep algorithm after initial testing that demonstrated MuDeep compared to other methods led to higher rank accuracies. This Jupyter Notebook provides an example of how we used MuDeep to evaluate the Eurecom + Iris (EI) favtGAN Baseline generated thermal faces. The experiment is entitled `eur_EI_GAND_data`. All data must be formatted in a specific format of `ID_CameraID.png`. Examples of 21 generated thermal images for Test Subject 001 is provided in the directory as an example. The preprocessing and preparation required is the following: 

1. Format filenames of generated thermal images for each experiment into the correct filename format. These will serve as your probes. 
2. Upload the `probes` and the `gallery` to the server that hosts the torchreid library in the `reid-data` directory (see below). Examples of a gallery images are also provided in this directory. Gallery images are the ground truth thermal images. We provided the 21 ground truth thermal images for Test Subject 001 in `gallery_samples`.
3. The `torchreid` repo should already be installed. Ensure you have created a directory called `reid-data`. Path: `deep-person-reid/reid-data`. In this `reid-data` dir, upload the probes and gallery. Also in this same directory make a copy of the `gallery` as a `train` directory. No training occurs, but this is required for the model evaluation.

In [1]:
import torchreid

In [2]:
from __future__ import division, print_function, absolute_import
import re
import glob
import os
import os.path as osp

<h2>EI_GAND</h2>

In [15]:
from eur_EI_GAND_data import Faces
exp = 'EI_GAND'

In [17]:
torchreid.data.register_image_dataset(exp, Faces) # this registered successfully

datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources= exp,
    targets= exp',
    height=256,
    width=128,
    batch_size_train=3,
    batch_size_test=3,
    transforms=['random_flip', 'random_crop']
)
model = torchreid.models.build_model(
    name='mudeep',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

engine.run(
    save_dir='log/mudeep',
    eval_freq=10,
    print_freq=10,
    test_only=True,
    visrank=True
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Here, at transform_te...
=> Loading train (source) dataset
=> Loaded Faces
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |    50 |     1050 |        21
  query    |     5 |      105 |        21
  gallery  |    50 |     1050 |        21
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded Faces
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |    50 |     1050 |        21
  query    |     5 |      105 | 